In [ ]:
#%pip install duckdb fsspec gcsfs

In [2]:
import duckdb
from fsspec import filesystem
import os

In [ ]:
cn = duckdb.connect()
cn.register_filesystem(filesystem('gcs'))
cn.execute("INSTALL ducklake")

In [4]:
GCS_BUCKET = os.environ.get('GCS_BUCKET')
cn.execute(f"ATTACH 'ducklake:gcs_flock.ducklake' as gcs_flock (DATA_PATH 'gs://{GCS_BUCKET}/gcs_flock')")
cn.execute("USE gcs_flock")

In [ ]:
cn.execute(f"ATTACH 'ducklake:local_flock.ducklake' as local_flock")

In [ ]:
sql = """
create or replace table gcs_flock.order_header 
as 
select 1 as order_id, date('2023-01-01') as order_dt, 1001 as cust_id, 250.75 as ord_amount
union all select 2, date('2023-01-02'), 1002, 150.50
union all select 3, date('2023-01-03'), 1003, 300.00
"""
cn.execute(sql)

In [6]:
sql = """
create or replace table local_flock.order_header_spi
as 
select 1 as order_id, 'John Doe' as cust_name, date('1987-01-01') as dob
union all select 2, 'Jane Doe', date('1992-03-05')
union all select 3, 'Tom Brady', date('1977-08-14')
"""
cn.execute(sql)

In [11]:
sql = """
update order_header
set ord_amount = ord_amount * 1.1
where order_id = 2
"""
cn.execute(sql)

In [7]:
cn.sql("from gcs_flock.order_header").show()

┌──────────┬────────────┬─────────┬──────────────┐
│ order_id │  order_dt  │ cust_id │  ord_amount  │
│  int32   │    date    │  int32  │ decimal(5,2) │
├──────────┼────────────┼─────────┼──────────────┤
│        1 │ 2023-01-01 │    1001 │       250.75 │
│        3 │ 2023-01-03 │    1003 │       300.00 │
│        2 │ 2023-01-02 │    1002 │       165.55 │
└──────────┴────────────┴─────────┴──────────────┘



In [9]:
sql = """
    select oh.order_id, oh.order_dt, oh.cust_id, oh.ord_amount, oh_spi.cust_name, oh_spi.dob
    from gcs_flock.order_header oh
    join local_flock.order_header_spi oh_spi
    on oh.order_id = oh_spi.order_id
    where oh.ord_amount > 200
    order by oh.order_id
"""
cn.sql(sql).show()

┌──────────┬────────────┬─────────┬──────────────┬───────────┬────────────┐
│ order_id │  order_dt  │ cust_id │  ord_amount  │ cust_name │    dob     │
│  int32   │    date    │  int32  │ decimal(5,2) │  varchar  │    date    │
├──────────┼────────────┼─────────┼──────────────┼───────────┼────────────┤
│        1 │ 2023-01-01 │    1001 │       250.75 │ John Doe  │ 1987-01-01 │
│        3 │ 2023-01-03 │    1003 │       300.00 │ Tom Brady │ 1977-08-14 │
└──────────┴────────────┴─────────┴──────────────┴───────────┴────────────┘

